https://moncoachdata.com/blog/groupby-de-pandas/

In [1088]:
import pandas as pd

df = pd.DataFrame({
   "nom": ["John", "Lydia", "Laury", "Arnaud", "Enzo"],
   "genre": ["Homme", "Femme", "Femme", "Homme", "Homme"],
   "taille": [182, 166, 170, 178, 172],
   "education": ["Master", "Master", "Licence", "Licence", "Master"],
   "salaire": [65000, 72000, 74000, 68000, 80000]
})
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,Homme,172,Master,80000


In [1089]:
# df.groupby("genre").mean() -> error

À l’avenir, il faudra spécifier à GroupBy de Pandas de spécifier uniquement les colonnes numériques ou sélectionner manuellement les colonnes (numériques) qui nous intéressent !

In [1090]:
# groupby(critère) puis sélection de colonnes avant aggrégation
df.groupby("genre")[['taille', 'salaire']].mean()

,taille,salaire
genre,,
Femme,168.000000,73000.0
Homme,177.333333,71000.0


Dans certains cas, nous ne voulons appliquer la fonction d’agrégation qu’à une colonne spécifique. Une façon de procéder consiste à filtrer les colonnes avant d’appliquer la fonction groupby.

In [1091]:
# sélection de colonnes puis groupby(critère)
df[["genre","salaire","taille"]].groupby("genre").mean()

,salaire,taille
genre,,
Femme,73000.0,168.000000
Homme,71000.0,177.333333


Ce que nous avons fait dans la deuxième étape n’est pas la solution optimale. Un ensemble de données typique de la vie réelle contient plusieurs colonnes et nous pouvons avoir besoin de calculer des agrégations non pas sur toutes les colonnes, mais sur un grand nombre d’entre elles.

Dans ce cas, le filtrage des colonnes est une tâche fastidieuse. Une meilleure méthode consiste à utiliser la fonction NamedAgg de Pandas.

In [1092]:
# aggregate (column, method)
df.groupby("genre").agg(salaire_moyen = pd.NamedAgg("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,71000.0


Il existe même une méthode plus simple que celle de la troisième étape. La syntaxe est la suivante :

In [1093]:
# or simply
df.groupby("genre").agg(salaire_moyen=("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,71000.0


Une fois les groupes générés, nous pouvons calculer autant d’agrégations que nécessaire. Par exemple, le code suivant calcule le salaire médian et la taille moyenne pour chaque sexe.

In [1094]:
df.groupby("genre").agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

,salaire_median,taille_moyenne
genre,,
Femme,73000.0,168.000000
Homme,68000.0,177.333333


Jusqu’à présent, les groupes sont affichés en tant qu’index d’un DataFrame. Dans certains cas, il est préférable de les afficher sous forme de colonne dans le DataFrame. Nous pouvons y parvenir en utilisant le paramètre as_index.


In [1095]:
df.groupby("genre", as_index=False).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

,genre,salaire_median,taille_moyenne
0,Femme,73000.0,168.000000
1,Homme,68000.0,177.333333


Tout comme nous pouvons calculer des agrégations sur plusieurs colonnes, nous pouvons créer des groupes basés sur plusieurs colonnes. Si nous transmettons deux colonnes à la fonction groupby, celle-ci crée des groupes basés sur la combinaison de valeurs distinctes dans chaque colonne.

groupby([ordered list of criters to group by levels])

In [1096]:
df.groupby(["genre","education"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

salaire_median  taille_moyenne
genre education                                
Femme Licence           74000.0           170.0
      Master            72000.0           166.0
Homme Licence           68000.0           178.0
      Master            72500.0           177.0

Order matters !

In [1097]:
df.groupby(["education", "genre"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

salaire_median  taille_moyenne
education genre                                
Licence   Femme         74000.0           170.0
          Homme         68000.0           178.0
Master    Femme         72000.0           166.0
          Homme         72500.0           177.0

Ce n’est pas nécessaire dans notre cas, mais lorsque nous avons plusieurs groupes, nous pouvons vouloir trier les groupes sur la base des valeurs agrégées. Supposons que nous ayons 100 colonnes et que nous devions trouver les 3 premiers groupes en termes de salaire moyen le plus élevé. Dans ce cas, il est nécessaire de trier les résultats.

La fonction sort_values peut être utilisée avec la fonction groupby comme suit :

In [1098]:
df.groupby(["genre","education"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
).sort_values(by="salaire_median", ascending=False)

,,salaire_median,taille_moyenne
genre,education,,
Femme,Licence,74000.0,170.0
Homme,Master,72500.0,177.0
Femme,Master,72000.0,166.0
Homme,Licence,68000.0,178.0


In [1099]:
df.groupby(["education", "genre"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
).sort_values(by="salaire_median", ascending=False)

salaire_median  taille_moyenne
education genre                                
Licence   Femme         74000.0           170.0
Master    Homme         72500.0           177.0
          Femme         72000.0           166.0
Licence   Homme         68000.0           178.0

Notre base de données ne contient pas de valeurs manquantes. Toutefois, les ensembles de données réels sont susceptibles d’en contenir. S’il y a des valeurs manquantes dans la colonne utilisée pour le regroupement, ces lignes sont supprimées par défaut.


In [1100]:
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,Homme,172,Master,80000


Commençons par mettre à jour une valeur manquante dans notre base de données.

In [1101]:
df.iloc[4,1] = None
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,None,172,Master,80000


In [1102]:
df.groupby("genre").agg(salaire_moyen=("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,66500.0


Comme tu peux le constater, la valeur manquante (None) dans la colonne ‘genre’ est ignorée. Cependant, il est important de prendre en compte les valeurs manquantes dans de nombreux cas. Nous pouvons définir le paramètre dropna sur False pour inclure les valeurs manquantes dans les résultats.

In [1103]:
df.groupby("genre", dropna=False).agg(salaire_moyen=("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,66500.0
NaN,80000.0


Nous avons vu comment calculer plusieurs agrégations à l’étape 5. Une autre méthode pour cette opération consiste à passer une liste d’agrégations à la fonction agg

In [1104]:
df[["genre", 'taille', 'salaire']].groupby("genre", dropna=False).agg(["mean","count"])

taille        salaire      
        mean count     mean count
genre                            
Femme  168.0     2  73000.0     2
Homme  180.0     2  66500.0     2
NaN    172.0     1  80000.0     1

https://pandas.pydata.org/docs/user_guide/groupby.html

In [1105]:
import numpy as np
speeds = pd.DataFrame(
   [
   ("bird", "Falconiformes", 389.0),
   ("bird", "Psittaciformes", 24.0),
   ("mammal", "Carnivora", 80.2),
   ("mammal", "Primates", np.nan),
   ("mammal", "Carnivora", 58),
   ],
   index=["falcon", "parrot", "lion", "monkey", "leopard"],
   columns=("class", "order", "max_speed"),
   )
    

speeds


,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [1106]:
grouped = speeds.groupby("class")
grouped["max_speed"].mean()

class
bird      206.5
mammal     69.1
Name: max_speed, dtype: float64

In [1107]:
grouped = speeds.groupby(["class", "order"])
grouped["max_speed"].mean()


class   order         
bird    Falconiformes     389.0
        Psittaciformes     24.0
mammal  Carnivora          69.1
        Primates            NaN
Name: max_speed, dtype: float64

In [1108]:
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)


df

,A,B,C,D
0,foo,one,1.227008,-0.849735
1,bar,one,-0.231442,-0.934994
2,foo,two,0.101682,0.937484
3,bar,three,-0.381073,0.304726
4,foo,two,-1.064318,-0.831111
5,bar,two,-0.273912,-0.665913
6,foo,one,-0.399356,-0.974197
7,foo,three,0.842800,1.003358


In [1109]:
grouped = df.groupby("A")
grouped[["C","D"]].mean()

,C,D
A,,
bar,-0.295476,-0.43206
foo,0.141563,-0.14284


In [1110]:

grouped = df.groupby("B")
grouped[["C","D"]].mean()


,C,D
B,,
one,0.198737,-0.919642
three,0.230863,0.654042
two,-0.412183,-0.186513


In [1111]:
grouped = df.groupby(["A", "B"])
grouped[["C","D"]].mean()

C         D
A   B                        
bar one   -0.231442 -0.934994
    three -0.381073  0.304726
    two   -0.273912 -0.665913
foo one    0.413826 -0.911966
    three  0.842800  1.003358
    two   -0.481318  0.053186

In [1112]:
df2 = df.set_index(["A", "B"])
df2

C         D
A   B                        
foo one    1.227008 -0.849735
bar one   -0.231442 -0.934994
foo two    0.101682  0.937484
bar three -0.381073  0.304726
foo two   -1.064318 -0.831111
bar two   -0.273912 -0.665913
foo one   -0.399356 -0.974197
    three  0.842800  1.003358

If we also have a MultiIndex on columns A and B, we can group by all the columns except the one we specify:

In [1113]:
# only A
grouped = df2.groupby(level=df2.index.names.difference(["B"]))
grouped[["C","D"]].sum()


,C,D
A,,
bar,-0.886427,-1.296180
foo,0.707815,-0.714202


In [1114]:
grouped.sum()

,C,D
A,,
bar,-0.886427,-1.296180
foo,0.707815,-0.714202


In [1115]:
# only B from multiindex dataframe
grouped = df2.groupby(level=df2.index.names.difference(["A"]))
grouped.mean()  # aggregate

,C,D
B,,
one,0.198737,-0.919642
three,0.230863,0.654042
two,-0.412183,-0.186513


The above GroupBy will split the DataFrame on its index (rows). To split by columns, first do a transpose:

In [1116]:
dfT = df.T

In [1117]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'


grouped = df.T.groupby(get_letter_type)
list(grouped)

[('consonant',
            0         1         2         3         4         5         6  \
  B       one       one       two     three       two       two       one   
  C  1.227008 -0.231442  0.101682 -0.381073 -1.064318 -0.273912 -0.399356   
  D -0.849735 -0.934994  0.937484  0.304726 -0.831111 -0.665913 -0.974197   
  
            7  
  B     three  
  C    0.8428  
  D  1.003358  ),
 ('vowel',
       0    1    2    3    4    5    6    7
  A  foo  bar  foo  bar  foo  bar  foo  foo)]

In [1118]:
#df.T.reset_index()
# Duplicate the index into a new column
dfT['Index_Column'] = dfT.index
dfT['Index_Column'] = dfT['Index_Column'].apply(get_letter_type)
dfT

,0,1,2,3,4,5,6,7,Index_Column
A,foo,bar,foo,bar,foo,bar,foo,foo,vowel
B,one,one,two,three,two,two,one,three,consonant
C,1.227008,-0.231442,0.101682,-0.381073,-1.064318,-0.273912,-0.399356,0.8428,consonant
D,-0.849735,-0.934994,0.937484,0.304726,-0.831111,-0.665913,-0.974197,1.003358,consonant


In [1119]:
dfT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, A to D
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   0             4 non-null      object
 1   1             4 non-null      object
 2   2             4 non-null      object
 3   3             4 non-null      object
 4   4             4 non-null      object
 5   5             4 non-null      object
 6   6             4 non-null      object
 7   7             4 non-null      object
 8   Index_Column  4 non-null      object
dtypes: object(9)
memory usage: 492.0+ bytes


In [1120]:
#dfT = dfT.set_index("Index_Column")

In [1081]:
grouped = dfT.groupby(['Index_Column']) 


In [1086]:
grouped[list(dfT.columns[[0]])].obj

,0,1,2,3,4,5,6,7,Index_Column
A,foo,bar,foo,bar,foo,bar,foo,foo,vowel
B,one,one,two,three,two,two,one,three,consonant
C,0.789325,0.586024,1.249318,1.010353,0.249099,-1.595121,1.256585,0.326607,consonant
D,-0.626654,-1.255759,-1.390153,0.179713,0.618323,1.030391,0.173211,-0.11551,consonant


In [896]:
grouped.obj.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 'Index_Column'], dtype='object')

In [897]:
grouped[grouped.obj.columns]

In [899]:
df2 = df.set_index(list(df.columns[[0]]))
df2

,B,C,D
A,,,
foo,one,-1.517662,-0.688679
bar,one,0.000049,0.806417
foo,two,-0.319354,-0.503013
bar,three,0.499718,-0.558471
foo,two,-0.395866,0.899330
bar,two,-1.161677,-0.217475
foo,one,1.507911,-0.807724
foo,three,0.238521,1.405253


In [900]:
df2.loc["consonant"]

KeyError: 'consonant'

In [780]:
# Sample DataFrame
data = {
    'Category': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Values1': [1, 2, 3, 4, 5, 6],
    'Values2': [10, 20, 30, 40, 50, 60]
}

df = pd.DataFrame(data)
print("Original DataFrame:")
print(df)

Original DataFrame:
  Category  Values1  Values2
0        A        1       10
1        A        2       20
2        B        3       30
3        B        4       40
4        C        5       50
5        C        6       60


In [781]:
# Grouping the DataFrame by the 'Category' column
grouped = df.groupby('Category')
grouped


In [782]:

# Accessing columns from the grouped object
columns = grouped.obj.columns
print("\nColumns in the grouped object:")
print(columns)


Columns in the grouped object:
Index(['Category', 'Values1', 'Values2'], dtype='object')


In [783]:
# Aggregating the grouped data (summing the values)
aggregated = grouped.sum()
print("\nAggregated DataFrame:")
print(aggregated)

# Accessing columns of the aggregated DataFrame
aggregated_columns = aggregated.columns
print("\nColumns in the aggregated DataFrame:")
print(aggregated_columns)


Aggregated DataFrame:
          Values1  Values2
Category                  
A               3       30
B               7       70
C              11      110

Columns in the aggregated DataFrame:
Index(['Values1', 'Values2'], dtype='object')
